Install dependencies



# New Section

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pyarrow
import os 
from datetime import datetime
from tqdm import tqdm

import gspread
from oauth2client.client import GoogleCredentials
from google.colab import auth, files

import ast
import json
import re

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

## 6 Month Sample Dataset Processing

In [ ]:
def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    try:
      return ast.literal_eval(d)
    except:
      return extract_from_lists(d)
    else:
      return None

def extract_from_lists(row):
  for elem in row:
    if elem == None: return (None)
    else: return dict(elem)

def list_of_dicts(ld, field):
    '''
    Create a mapping of the tuples formed after 
    converting json strings of list to a python list   
    '''
    try:
      result = []
      for d in ast.literal_eval(ld):
        if d == None:
          continue
        else:
          result.append(d[field])
      return result
    except (TypeError, ValueError):
      return only_dict(ld)[field]
    else:
      return 'unable to retrieve'

def data_from_json(data, field):
  """Using the rows from the column and the field wanting to be selected
  the function finds the best method to parse the json code and return
  the nested value."""
  from json import JSONDecodeError

  try:
    json_data = json.dumps(data)
    json_field = json.loads(json_data.replace('\'', '\"')[1:-1])[field]
    return json_field
  except TypeError:
    return only_dict(data)[field]
  except JSONDecodeError:
    json_data = json.dumps(data)
    return list_of_dicts(json.loads(json_data), field)
  else:
    return None

def get_selected_columns(filename='/content/drive/Shareddrives/SIADS - 694-695 Team Drive/Notebooks/final_columns.txt'):
  """
  filename: """
  columns = []
  with open(filename, 'r') as f:
    for ln in f.readlines():
      col = (ln.strip().split('\xa0\xa0')[1])
      groups = re.search(r"'(\w.+)'", col)
      columns.append(groups.group(1))
  return columns

def preprocessing(sample, columns:list):
  """
  Given a sample DF, expand columns passed in args.
  """
  drop_cols = set()

  for col in tqdm(columns):
    try:
      splits = col.split('.')
      colName = ".".join(splits)
      if colName in sample.columns:
        continue
      elif len(splits) == 2:
        sample[colName] = sample[splits[0]].apply(lambda x: data_from_json(x, splits[1]))
      elif len(splits) == 3:
        prefix = ".".join(splits[:2])
        if prefix in sample.columns:
          sample[colName] = sample[prefix].apply(lambda x: data_from_json(x, splits[2]))
        else:
          sample[prefix] = sample[splits[0]].apply(lambda x: data_from_json(x, splits[1]))
          sample[colName] = sample[prefix].apply(lambda x: data_from_json(x, splits[2]))
        drop_cols.add(splits[0])
        drop_cols.add(prefix)
    except:
      splits = col.split('.')
      colName = ".".join(splits)
      if len(splits) == 3:
        prefix = ".".join(splits[:2])
        # TODO: Chnage this to input sample df
        processed_data[colName] = \
        processed_data[prefix].apply(lambda x: only_dict(x)[splits[2]] if only_dict(x) is not None else None)
    else:
      continue
  
  return sample, drop_cols

In [ ]:
sample_dataset = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/sample_file.csv')

In [ ]:
columns = get_selected_columns()
print(columns)

['date', 'totals', 'fullVisitorId', 'clientId', 'totals.visits', 'totals.hits', 'totals.pageviews', 'totals.timeOnSite', 'totals.bounces', 'totals.transactions', 'totals.newVisits', 'totals.screenviews', 'totals.uniqueScreenviews', 'totals.timeOnScreen', 'totals.totalTransactionRevenue', 'hits.type', 'hits', 'channelGrouping', 'socialEngagementType', 'geoNetwork.country', 'trafficSource.source', 'trafficSource.medium', 'trafficSource.isTrueDirect', 'device.browser', 'device.operatingSystem', 'device.deviceCategory', 'hits.hour', 'hits.minute', 'hits.transaction', 'hits.item', 'hits.dataSource', 'hits.transaction.transactionRevenue', 'hits.item.productName', 'hits.item.productCategory', 'hits.item.itemRevenue', 'hits.appInfo.screenDepth', 'hits.eCommerceAction.action_type']


In [ ]:
processed_data, drop_cols = preprocessing(sample_dataset, columns)

100%|██████████| 37/37 [56:00<00:00, 90.82s/it] 


In [ ]:
remaining = list(set(columns).difference(set(processed_data.columns)))
for col in remaining:
  print(col)

clientId


In [ ]:
processed_data, drop_cols = preprocessing(processed_data, remaining)

100%|██████████| 37/37 [00:02<00:00, 14.18it/s]


In [ ]:
processed_data.to_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/processed_file.csv')

In [ ]:
processed_data.head(2)

,socialEngagementType,hits,channelGrouping,trafficSource,totals,date,device,fullVisitorId,geoNetwork,totals.visits,...,hits.dataSource,hits.appInfo,hits.eCommerceAction,totals.bounces,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.productCategory,hits.item.itemRevenue
0,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",Social,"{'referralPath': '/', 'campaign': '(not set)',...","{'visits': 1, 'hits': 10, 'pageviews': 9, 'tim...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",5160807529212499245,"{'continent': 'Europe', 'subContinent': 'Easte...",1,...,"[None, None, None, None, None, None, None, Non...","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,None,None,None,None
1,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 22, 'minu...",Organic Search,"{'referralPath': None, 'campaign': '(not set)'...","{'visits': 1, 'hits': 2, 'pageviews': 2, 'time...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",558025075151655343,"{'continent': 'Americas', 'subContinent': 'Nor...",1,...,"[None, None]","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,None,None,None,None


## Examine Saved File 

In [ ]:
saved = pd.read_csv('/content/drive/Shareddrives/SIADS - 694-695 Team Drive/datasets/processed_file.csv', index_col=0)

In [ ]:
remaining = list(set(columns).difference(set(saved.columns)))
remaining

['clientId']

In [ ]:
saved.shape

(50106, 41)

In [ ]:
from collections import Counter
core = [col for col in saved.columns if '.' not in col]
additional = [col for col in saved.columns if '.' in col]
second = []
third = []

for col in additional:
  count = Counter(col)
  if count['.'] == 1:
    second.append(col)
  elif count['.'] == 2:
    third.append(col)

In [ ]:
newly_processed = saved.copy()
for col in additional:
  newly_processed = newly_processed.explode(col)

In [ ]:
newly_processed.head(1)

,socialEngagementType,hits,channelGrouping,trafficSource,totals,date,device,fullVisitorId,geoNetwork,totals.visits,...,hits.dataSource,hits.appInfo,hits.eCommerceAction,totals.bounces,hits.appInfo.screenDepth,hits.eCommerceAction.action_type,hits.transaction.transactionRevenue,hits.item.productName,hits.item.productCategory,hits.item.itemRevenue
0,Not Socially Engaged,"[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",Social,"{'referralPath': '/', 'campaign': '(not set)',...","{'visits': 1, 'hits': 10, 'pageviews': 9, 'tim...",20160801,"{'browser': 'Chrome', 'browserVersion': 'not a...",5160807529212499245,"{'continent': 'Europe', 'subContinent': 'Easte...",1,...,"[None, None, None, None, None, None, None, Non...","[{'name': None, 'version': None, 'id': None, '...","[{'action_type': '0', 'step': 1, 'option': Non...",NaN,0,0,NaN,NaN,NaN,NaN


drive  sample_data
